In [1]:
from functions_for_pred import *

[nltk_data] Downloading package wordnet to /Users/daisy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/daisy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
test = pd.read_csv('data/test.csv')
X_valid = pd.read_csv('test/X_valid.csv')
y_valid = pd.read_csv('test/y_valid.csv')
X_train = pd.read_csv('test/X_train.csv')
y_train = pd.read_csv('test/y_train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
X_train = X_train.drop(columns=['Unnamed: 0'])
X_valid = X_valid.drop(columns=['Unnamed: 0'])

In [4]:
## prepare for numeric values preparation
pca = pickle.load(open("model/pca.pkl", "rb"))
lda = pickle.load(open("model/lda.pkl", "rb"))
vect_col = pickle.load(open("model/vect_col.pkl", "rb"))
num_pred_col = pickle.load(open("model/numeric_train_col.pkl","rb"))

X_train = create_pca_lda_for_test(X_train, vect_col)
X_train2 = create_new_features(X_train)

X_valid = create_pca_lda_for_test(X_valid, vect_col)
X_valid2 = create_new_features(X_valid)


In [5]:
#check prediction
xgb_model_num = XGBClassifier()
xgb_model_num.load_model('model/xgb_model_num.model')

In [6]:
xgb_model_num.predict(X_valid2[num_pred_col])

array(['folk', 'folk', 'classic pop and rock', ..., 'pop',
       'dance and electronica', 'folk'], dtype='<U21')

In [7]:
## for text classifications preparation

lda_model = gensim.models.ldamulticore.LdaMulticore.load('model/lda_model.model')
train_id2word =  pickle.load(open("model/train_id2word.pkl", 'rb'))
lda_model2 = gensim.models.ldamulticore.LdaMulticore.load('model/lda_model2.model')

X_train_text_array1, y_train_text_array1 = prepare_text_for_prediction(X_train, y_train, train_id2word,lda_model)
X_train_text_array2, y_train_text_array2 = prepare_text_for_prediction(X_train, y_train, train_id2word,lda_model2)

X_text_array1, y_text_array1 = prepare_text_for_prediction(X_valid, y_valid, train_id2word,lda_model)
X_text_array2, y_text_array2 = prepare_text_for_prediction(X_valid, y_valid, train_id2word,lda_model2)

In [8]:
xgb_model_text= XGBClassifier()
xgb_model_text.load_model('model/xgb_model_text.model')

In [9]:
train_vecs=np.concatenate( [X_train_text_array1, X_train_text_array2], axis=1)

valid_vecs = np.concatenate([X_text_array1, X_text_array2], axis=1)
X_valid_vecs = np.array(valid_vecs)
text_pred = xgb_model_text.predict(X_valid_vecs)

In [10]:
expected_pred = pd.read_csv('test/valid_test_prediction.csv')

In [11]:
sum(expected_pred['pred']!=text_pred)

256

In [12]:
X_train2=X_train2.drop(columns=['text'])
X_valid2=X_valid2.drop(columns=['text'])

In [13]:
np.shape(X_train2),

((4876, 172),)

In [14]:
X_train_final = np.concatenate([train_vecs,np.array(X_train2)],axis=1)
X_valid_final = np.concatenate([valid_vecs,np.array(X_valid2)],axis=1)

In [15]:
np.shape(X_valid_final)

(3252, 190)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
X_valid_1, X_test_1, y_valid_1, y_test_1 = train_test_split(X_valid_final , y_valid, test_size=0.5, random_state=123)


In [20]:
max_depth = [3, 5, 7]  #, 7
eta = [.2,.15] #  .1, .2,
gamma = [.25, .2] 
sampling_method = ['gradient_based']  
reg_lambda = [.2, .15]
colsample_bytree = [.35, .45] #.4,
alphax = [.5, .6] 
num_parallel_tree = [5] 

grid_para_2 = dict(max_depth=max_depth, reg_lambda=reg_lambda, reg_alpha=alphax,
                 colsample_bytree=colsample_bytree, eta=eta, gamma=gamma,
                 sampling_method=sampling_method, 
                 num_parallel_tree=num_parallel_tree)


In [21]:
def grid_search_CV(grid_para, X, y, test_X, test_y, name):

    model_1 = GridSearchCV(XGBClassifier(eval_metric='mlogloss'),
                               grid_para, cv=3)
    model_1.fit(X, y, eval_set = [(test_X, test_y)],
                early_stopping_rounds = 10)
    print(model_1.best_score_)
    print(model_1.best_params_)
   
    xgb_m = XGBClassifier(eval_metric = ['mlogloss'],
                          objective='multi:softprob',
                          booster = 'gbtree',
                          grow_policy='lossguide',
                          max_depth=model_1.best_params_['max_depth'],
                          reg_alpha=model_1.best_params_['reg_alpha'],
                          reg_lambda=model_1.best_params_['reg_lambda'],
                          eta=model_1.best_params_['eta'],
                          num_parallel_tree = model_1.best_params_['num_parallel_tree'],
                          gamma=model_1.best_params_['gamma'],
                          sampling_method=model_1.best_params_['sampling_method'],
                          colsample_bytree=model_1.best_params_['colsample_bytree'],
                          seed=123,
                          n_jobs = -1,
                            )
    xgb_m.fit(X, y)
    xgb_m.save_model('model/xgb_model_' + name + '.model')
    return xgb_m


In [ ]:
xgb_model = grid_search_CV(grid_para_2, X_train_final, y_train['genre'], X_valid_1, y_valid_1['genre'],'all') #



[0]	validation_0-mlogloss:1.89088


/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.76590
[2]	validation_0-mlogloss:1.67088
[3]	validation_0-mlogloss:1.59444
[4]	validation_0-mlogloss:1.52758
[5]	validation_0-mlogloss:1.47282
[6]	validation_0-mlogloss:1.42725
[7]	validation_0-mlogloss:1.38935
[8]	validation_0-mlogloss:1.35703
[9]	validation_0-mlogloss:1.32798
[10]	validation_0-mlogloss:1.30310
[11]	validation_0-mlogloss:1.28128
[12]	validation_0-mlogloss:1.26041
[13]	validation_0-mlogloss:1.24256
[14]	validation_0-mlogloss:1.22560
[15]	validation_0-mlogloss:1.21062
[16]	validation_0-mlogloss:1.19789
[17]	validation_0-mlogloss:1.18642
[18]	validation_0-mlogloss:1.17614
[19]	validation_0-mlogloss:1.16616
[20]	validation_0-mlogloss:1.15735
[21]	validation_0-mlogloss:1.14907
[22]	validation_0-mlogloss:1.14102
[23]	validation_0-mlogloss:1.13344
[24]	validation_0-mlogloss:1.12685
[25]	validation_0-mlogloss:1.12027
[26]	validation_0-mlogloss:1.11450
[27]	validation_0-mlogloss:1.10914
[28]	validation_0-mlogloss:1.10426
[29]	validation_0-mlogloss:1.

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.76696
[2]	validation_0-mlogloss:1.66859
[3]	validation_0-mlogloss:1.59456
[4]	validation_0-mlogloss:1.52867
[5]	validation_0-mlogloss:1.47372
[6]	validation_0-mlogloss:1.42904
[7]	validation_0-mlogloss:1.39045
[8]	validation_0-mlogloss:1.35725
[9]	validation_0-mlogloss:1.32689
[10]	validation_0-mlogloss:1.30128
[11]	validation_0-mlogloss:1.27993
[12]	validation_0-mlogloss:1.25846
[13]	validation_0-mlogloss:1.24083
[14]	validation_0-mlogloss:1.22355
[15]	validation_0-mlogloss:1.20823
[16]	validation_0-mlogloss:1.19495
[17]	validation_0-mlogloss:1.18262
[18]	validation_0-mlogloss:1.17139
[19]	validation_0-mlogloss:1.16008
[20]	validation_0-mlogloss:1.15146
[21]	validation_0-mlogloss:1.14253
[22]	validation_0-mlogloss:1.13395
[23]	validation_0-mlogloss:1.12615
[24]	validation_0-mlogloss:1.11867
[25]	validation_0-mlogloss:1.11275
[26]	validation_0-mlogloss:1.10668
[27]	validation_0-mlogloss:1.10139
[28]	validation_0-mlogloss:1.09631
[29]	validation_0-mlogloss:1.

...

[0]	validation_0-mlogloss:1.83679
[1]	validation_0-mlogloss:1.68831
[2]	validation_0-mlogloss:1.58105
[3]	validation_0-mlogloss:1.49686
[4]	validation_0-mlogloss:1.42627
[5]	validation_0-mlogloss:1.37005
[6]	validation_0-mlogloss:1.32175
[7]	validation_0-mlogloss:1.28134
[8]	validation_0-mlogloss:1.24688
[9]	validation_0-mlogloss:1.21781
[10]	validation_0-mlogloss:1.19244
[11]	validation_0-mlogloss:1.17145
[12]	validation_0-mlogloss:1.15158
[13]	validation_0-mlogloss:1.13484
[14]	validation_0-mlogloss:1.12008
[15]	validation_0-mlogloss:1.10706
[16]	validation_0-mlogloss:1.09557
[17]	validation_0-mlogloss:1.08627
[18]	validation_0-mlogloss:1.07641
[19]	validation_0-mlogloss:1.06821
[20]	validation_0-mlogloss:1.06047
[21]	validation_0-mlogloss:1.05301
[22]	validation_0-mlogloss:1.04758
[23]	validation_0-mlogloss:1.04138
[24]	validation_0-mlogloss:1.03519
[25]	validation_0-mlogloss:1.03071
[26]	validation_0-mlogloss:1.02756
[27]	validation_0-mlogloss:1.02428
[28]	validation_0-mlogloss:1.0

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.75876
[2]	validation_0-mlogloss:1.65969
[3]	validation_0-mlogloss:1.58143
[4]	validation_0-mlogloss:1.51311
[5]	validation_0-mlogloss:1.45966
[6]	validation_0-mlogloss:1.41423
[7]	validation_0-mlogloss:1.37723
[8]	validation_0-mlogloss:1.34454
[9]	validation_0-mlogloss:1.31524
[10]	validation_0-mlogloss:1.28974
[11]	validation_0-mlogloss:1.26837
[12]	validation_0-mlogloss:1.24859
[13]	validation_0-mlogloss:1.23204
[14]	validation_0-mlogloss:1.21608
[15]	validation_0-mlogloss:1.20132
[16]	validation_0-mlogloss:1.18982
[17]	validation_0-mlogloss:1.17798
[18]	validation_0-mlogloss:1.16725
[19]	validation_0-mlogloss:1.15702
[20]	validation_0-mlogloss:1.14826
[21]	validation_0-mlogloss:1.14035
[22]	validation_0-mlogloss:1.13264
[23]	validation_0-mlogloss:1.12570
[24]	validation_0-mlogloss:1.11795
[25]	validation_0-mlogloss:1.11119
[26]	validation_0-mlogloss:1.10521
[27]	validation_0-mlogloss:1.10009
[28]	validation_0-mlogloss:1.09548
[29]	validation_0-mlogloss:1.

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.76290
[2]	validation_0-mlogloss:1.66383
[3]	validation_0-mlogloss:1.58550
[4]	validation_0-mlogloss:1.51921
[5]	validation_0-mlogloss:1.46511
[6]	validation_0-mlogloss:1.42070
[7]	validation_0-mlogloss:1.38143
[8]	validation_0-mlogloss:1.34889
[9]	validation_0-mlogloss:1.32029
[10]	validation_0-mlogloss:1.29344
[11]	validation_0-mlogloss:1.27095
[12]	validation_0-mlogloss:1.25003
[13]	validation_0-mlogloss:1.23096
[14]	validation_0-mlogloss:1.21476
[15]	validation_0-mlogloss:1.19936
[16]	validation_0-mlogloss:1.18645
[17]	validation_0-mlogloss:1.17431
[18]	validation_0-mlogloss:1.16341
[19]	validation_0-mlogloss:1.15324
[20]	validation_0-mlogloss:1.14374
[21]	validation_0-mlogloss:1.13595
[22]	validation_0-mlogloss:1.12776
[23]	validation_0-mlogloss:1.11950
[24]	validation_0-mlogloss:1.11261
[25]	validation_0-mlogloss:1.10632
[26]	validation_0-mlogloss:1.09969
[27]	validation_0-mlogloss:1.09438
[28]	validation_0-mlogloss:1.08931
[29]	validation_0-mlogloss:1.

In [ ]:
np.shape(X_valid_1),np.shape(X_test_1)

In [ ]:
 y_test_1['genre']

In [ ]:
ax= plt.subplot()
cm = confusion_matrix(xgb_model.predict(X_test_1), y_test_1['genre'])
sns.heatmap(cm, annot=True, fmt='g', ax=ax, cmap='Blues'); 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 


In [ ]:
print(classification_report(xgb_model.predict(X_test_1), y_test_1['genre']))

In [ ]:
print(cohen_kappa_score(xgb_model.predict(X_test_1), y_test_1['genre']))

In [ ]:
## for test predition
# prepare for numeric and text data
test = combine_title_and_tags(test)
X_test_num = create_pca_lda_for_test(test, vect_col)
X_test_num2 = create_new_features(X_test_num)


In [ ]:
X_test_num2 = X_test_num2.drop(columns=['text'])
X_test_text_array1, _ = prepare_text_for_prediction(test, np.nan, train_id2word,lda_model)
X_test_text_array2, _ = prepare_text_for_prediction(test, np.nan, train_id2word,lda_model2)
final_test_vecs=np.concatenate( [X_test_text_array1, X_test_text_array2], axis=1)
X_final_feature = np.concatenate([final_test_vecs,np.array(X_test_num2[num_pred_col])],axis=1)

In [ ]:
np.shape(X_final_feature)

In [ ]:
np.shape(valid_vecs), np.shape(final_test_vecs)

In [ ]:
pd.DataFrame(xgb_model.predict(X_final_feature),columns=['predictions']).to_csv('predictions/prediction.csv')

In [ ]:
np.shape(X_test_text_array2), np.shape(final_test_vecs)

In [ ]:
test = pd.read_csv('data/test.csv')

In [ ]:
test.dtypes

In [ ]:
types_dict = {}

In [ ]:
for i,j in zip(test.columns, test.dtypes):
    types_dict[i]=j

In [ ]:
with open('model/type_dict.pkl', 'wb') as f:
    pickle.dump(types_dict, f)